In [1]:
import pickle,os
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import scipy.signal as signal
import matplotlib.gridspec as gridspec
from itertools import chain
#import seaborn as sns
import networkx as nx
#sns.reset_defaults()

In [21]:
pathA="processedData/control/"
channel=pickle.load(open("pickles/channel.pickle","rb"))
locations=pickle.load(open("pickles/locations.pickle","rb"))
threshold=0.7

cluster=OrderedDict()
cluster['C']=['C{}'.format(i) for i in range(1,7)]
cluster['F']=['F{}'.format(i) for i in range(1,9)]
cluster['O']=['O{}'.format(i) for i in range(1,3)]
cluster['P']=['P{}'.format(i) for i in range(1,9)]

cluster['C'].extend(['CZ'])
cluster['F'].extend(['FZ'])
cluster['O'].extend(['OZ'])
cluster['P'].extend(['PZ'])


cluster['T']=['T7','T8']

studiedSensors=list(cluster.keys())

In [22]:
sensorMap={}
sensorMapRev={}
count=0
for sensor in channel:
    sensorMap[sensor]=count
    sensorMapRev[count]=sensor
    count+=1 

In [23]:
def initConnectionMatrix():
    matrix={}
    for i in range(len(cluster)):
        for j in range(0,i+1):
            key=studiedSensors[i]+"-"+studiedSensors[j]
            matrix[key]=0
    return matrix

def validateSensors(i,j):
    sensorA=sensorMapRev[i]
    sensorB=sensorMapRev[j]
    if sensorA[0] in studiedSensors and sensorB[0] in studiedSensors:
        if sensorA in cluster[sensorA[0]] and sensorB in cluster[sensorB[0]]:
            return True
        else:
            return False
    else:
            return False
        
def addConnection(trail,stimulus,i,j):
    sensorA=sensorMapRev[i]
    sensorB=sensorMapRev[j]
    try:
        key=sensorA[0]+'-'+sensorB[0]
        sumConnections[stimulus][key]+=1
        trailMatrix[trail][key]+=1
    except:
        key=sensorB[0]+'-'+sensorA[0]
        sumConnections[stimulus][key]+=1  
        trailMatrix[trail][key]+=1
        
def savitzkyGolay(data, parameters):
    #print(parameters)
    smooth_data = signal.savgol_filter(data, parameters[0], parameters[1])
    peel = np.array(data[-len(smooth_data):]) - np.array(smooth_data)
    return smooth_data, peel

In [24]:

stimuli=['S1obj','S2match','S2nomatch']

In [25]:
wins=[15,31,65,151,191]
bwaves=["Gamma","Beta","Alpha","Theta","Delta"]
# wins=[41]
# bwaves=["Peel"]
for winIndex, bwave in zip(range(len(wins)),bwaves):
    trails=os.listdir(pathA)
    clusterMatrix=initConnectionMatrix()
    trailMatrix=OrderedDict()
    
    ##
    sumConnections={}
    stimulusFreq={}
    for stimulus in stimuli:
        sumConnections[stimulus]=initConnectionMatrix()
        stimulusFreq[stimulus]=0
    ##
    
    for trail in trails:
        patient,stimulus=chain(trail.split("_"))
        stimulusFreq[stimulus]+=1
        data=pickle.load(open(pathA+trail,"rb"))

            
        tempdata=[]
        trailMatrix[trail]=initConnectionMatrix()
        for sensor in channel:
            tempdata.append(data[sensor])
            
        if winIndex==0:
            sdata,peeldata=savitzkyGolay(tempdata,[wins[winIndex],3]) 
        else:
            sdata,a=savitzkyGolay(tempdata,[wins[winIndex-1],3]) 
            sdata,b=savitzkyGolay(tempdata,[wins[winIndex],3]) 
            peeldata=np.array(b)-np.array(a)
            
        corrMatrix=np.corrcoef(peeldata)

        for i in range(len(tempdata)):
            for j in range(0,i):
                if validateSensors(i,j):
                    if corrMatrix[i][j]>=threshold:
                        addConnection(trail,stimulus,i,j)     
                        
    zMatrix=initConnectionMatrix()
    activation={}
    for stimulus in stimuli:
        activation[stimulus]=initConnectionMatrix()

    for conn in zMatrix:
        if conn[0]==conn[-1]:
            l=len(cluster[conn[0]])
            zMatrix[conn]=(l*(l-1))/2
        else:
            zMatrix[conn]=len(cluster[conn[0]])*len(cluster[conn[-1]])


    for stimulus in stimuli:
        n=stimulusFreq[stimulus]
        for conn in sumConnections[stimulus]:
            z=zMatrix[conn]
            value=sumConnections[stimulus][conn]
            activation[stimulus][conn]=value/(z*n)

    pickle.dump(activation,open("activation/brainWaves/control_{}Activation.pickle".format(bwave),"wb"))
    #pickle.dump(activation,open("activation/alcohol_{}Activation0.9.pickle".format(bwave),"wb"))
    activationStd={}
    for stimulus in stimuli:
        activationStd[stimulus]=initConnectionMatrix()
    t={}
    
    for trails in trailMatrix:
        trail,stimulus=chain(trails.split('_'))
        if stimulus not in t:
            t[stimulus]={}
        for cons in trailMatrix[trails]:
            if cons not in t[stimulus]:
                t[stimulus][cons]=[]
            t[stimulus][cons].append(trailMatrix[trails][cons]/zMatrix[cons])
        
    for stimulus in t:
        for cons in t[stimulus]:
            activation[stimulus][cons]=np.mean(t[stimulus][cons])
            activationStd[stimulus][cons]=np.std(t[stimulus][cons]) 
    #print(activationStd)
    pickle.dump(activationStd,open("activation/brainWaves/control_{}ActivationStd.pickle".format(bwave),"wb"))
    #pickle.dump(activationStd,open("activation/alcohol_{}ActivationStd0.9.pickle".format(bwave),"wb"))

c:\program files (x86)\python36-32\lib\site-packages\numpy\lib\function_base.py:3167: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\program files (x86)\python36-32\lib\site-packages\numpy\lib\function_base.py:3168: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [21]:
stimulusFreq

{'S1obj': 45, 'S2match': 45, 'S2nomatch': 45}

In [36]:
activationStd

{'S1obj': {'C-C': 0.092457951994932411,
  'F-C': 0.062657656478677076,
  'F-F': 0.22298394826252974,
  'O-C': 0.05254903288014872,
  'O-F': 0.0,
  'O-O': 0.24500496901542562,
  'P-C': 0.060084950546363543,
  'P-F': 0.0065269559689274722,
  'P-O': 0.25536430157454276,
  'P-P': 0.12748071869174926,
  'T-C': 0.11291059837319833,
  'T-F': 0.10485835737079541,
  'T-O': 0.080805274923227505,
  'T-P': 0.098099607532439154,
  'T-T': 0.14740554623801777},
 'S2match': {'C-C': 0.090127425819545331,
  'F-C': 0.030637108642833814,
  'F-F': 0.14522055922465354,
  'O-C': 0.01555231582719478,
  'O-F': 0.016378394026446419,
  'O-O': 0.26810312697561273,
  'P-C': 0.040516803638257451,
  'P-F': 0.018247779586629233,
  'P-O': 0.24814507706471334,
  'P-P': 0.11711488789614143,
  'T-C': 0.099203174552379314,
  'T-F': 0.059027323890774919,
  'T-O': 0.034346735168502604,
  'T-P': 0.080256409694543229,
  'T-T': 0.14740554623801777},
 'S2nomatch': {'C-C': 0.093935835010252461,
  'F-C': 0.053957878450717311,
  '

In [12]:
activation

{'S1obj': {'C-C': 0.3322751322751323,
  'F-C': 0.26208112874779543,
  'F-F': 0.6407407407407407,
  'O-C': 0.27724867724867724,
  'O-F': 0.13497942386831277,
  'O-O': 0.9037037037037037,
  'P-C': 0.27724867724867724,
  'P-F': 0.13058984910836763,
  'P-O': 0.6930041152263374,
  'P-P': 0.6666666666666666,
  'T-C': 0.41904761904761906,
  'T-F': 0.3469135802469136,
  'T-O': 0.3111111111111111,
  'T-P': 0.3037037037037037,
  'T-T': 0.4666666666666667},
 'S2match': {'C-C': 0.3513227513227513,
  'F-C': 0.2804232804232804,
  'F-F': 0.6574074074074074,
  'O-C': 0.2708994708994709,
  'O-F': 0.12757201646090535,
  'O-O': 0.8814814814814815,
  'P-C': 0.2948853615520282,
  'P-F': 0.11989026063100137,
  'P-O': 0.6625514403292181,
  'P-P': 0.6376543209876543,
  'T-C': 0.4492063492063492,
  'T-F': 0.30864197530864196,
  'T-O': 0.3925925925925926,
  'T-P': 0.3493827160493827,
  'T-T': 0.37777777777777777},
 'S2nomatch': {'C-C': 0.3386243386243386,
  'F-C': 0.2938271604938272,
  'F-F': 0.6641975308641975

In [27]:
a,b=chain([1,2])

In [28]:
a

1

In [42]:
for i in range(len(tempdata)):
        for j in range(0,i):
            print(i,j)

1 0
2 0
2 1
3 0
3 1
3 2
4 0
4 1
4 2
4 3
5 0
5 1
5 2
5 3
5 4
6 0
6 1
6 2
6 3
6 4
6 5
7 0
7 1
7 2
7 3
7 4
7 5
7 6
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
15 10
15 11
15 12
15 13
15 14
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
17 0
17 1
17 2
17 3
17 4
17 5
17 6
17 7
17 8
17 9
17 10
17 11
17 12
17 13
17 14
17 15
17 16
18 0
18 1
18 2
18 3
18 4
18 5
18 6
18 7
18 8
18 9
18 10
18 11
18 12
18 13
18 14
18 15
18 16
18 17
19 0
19 1
19 2
19 3
19 4
19 5
19 6
19 7
19 8
19 9
19 10
19 11
19 12
19 13
19 14
19 15
19 16
19 17
19 18
20 0
20 1
20 2
20 3
20 4
20 5
20 6
20 7
20 8
20 9
